---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
# importing packages
import pandas as pd
import numpy as np
import re 

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc,columns=['dates'])#converting the list to a dataframe

In [2]:
def convert(x):
    """Matches differernt patterns of dates
    """
    y = re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',x)#basic pattern
    if (len(y) == 0):
        #matching dates with month names and dotes in it 
        y = re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:\W)? (?:\d{1,2}\W |\d{1,2} )?(?:\d{2,4})',x)
        if(len(y) == 0):
            #matches dates with month names with ',' after the day
            y = re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}, )?(?:\d{2,4})',x)
            if(len(y) == 0):
                #matches dates with no days and dates with no days or months
                y= re.findall(r'(?:\d{1,2}[/-])?(?:\d{4})',x)
                if(len(y) == 0):return np.nan
                else:return y[0]
            else:return y[0]
        else: return y[0]
        
    else:return y[0]

In [3]:
def clean_dates(x):
    """function that match the month name and replace it with its number
    """
    y = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*',x)
    months_abb = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    months = ['January','February','March','April','May','June','July','August','September','October','November','December']
    wrong_mon = ['Janaury','February','March','April','May','June','July','August','September','October','November','Decemeber']
    repl = ''
    for i,j in enumerate(zip(months_abb,months,wrong_mon)):
        if(len(y) == 0):return x
        elif(y[0] in j):
            repl = '{}'.format(i+1)
            break
    string = re.sub(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*',repl,x)
    return string

In [4]:
def correct_years(x):
    """Function that add '19' to the years with just two digits
    """
    char = x.split('-')[-1]
    if(len(char) == 2):
        char = x.split('-')
        char[-1] = '19' + char[-1]
        string = '-'.join(char)
        return string
    else:return x

In [5]:
def date_sorter():
    """Function that combines all the previous together
    """
    
    df['order'] = df['dates'].apply(convert)
    df['order'] = df['order'].str.replace('/','-')
    df['order'] = df['order'].str.replace(' ','-')
    df['order'] = df['order'].str.replace(',','')
    df['order'] = df['order'].str.replace('.','')
    df['order'] = df['order'].apply(clean_dates)
    df['order'] = df['order'].apply(correct_years)
    format_1_1 = pd.to_datetime(df['order'].iloc[0:125],format='%m-%d-%Y')
    format_1_2 = pd.to_datetime(df['order'].iloc[195:228],format='%m-%d-%Y')
    format_2 = pd.to_datetime(df['order'].iloc[125:195],format='%d-%m-%Y')
    format_3 = pd.to_datetime(df['order'].iloc[228:455],format='%m-%Y')
    format_4 = pd.to_datetime(df['order'].iloc[455:500],format='%Y')
    df['order'] = pd.concat([format_1_1,format_2,format_1_2,format_3,format_4],axis=0)
    final = pd.Series(df['order'].sort_values().index)
    
    return final